In [ ]:
import pkg_resources

#pkg_resources.require("numpy==1.21")

import numpy as np
import matplotlib.pylab as plt
import uproot
import awkward as ak

import vector

import sys

vector.register_awkward()

#from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import coffea

from coffea.nanoevents import NanoEventsFactory
#from coffea.nanoevents import NanoAODSchema


In [ ]:
print(f"python: {sys.version}\n")

print(f"numpy:   {np.__version__}")
print(f"uproot:  {uproot.__version__}")
print(f"awkward: {ak.__version__}")
print(f"vector:  {vector.__version__}")
print()

print(f"coffea:  {coffea.__version__}")



In [ ]:
data_dir = '/home/bellis/top_data/NANOAOD/'

#infilename = 'small_skims_10k/TT_TToBCE_TuneCP5_BNV_2018_SMALL_10k.root'
infilename = 'TTToHadronic_UL_2018_SMALL_100k.root'

infile = uproot.open(data_dir + infilename)


In [ ]:
#events = coffea.nanoevents.NanoEventsFactory.from_root(infilename, schemaclass=NanoAODSchema).events()


In [ ]:
#infile['Events'].keys()

In [ ]:
jet_branch_arrays = infile["Events"].arrays(filter_name="Jet_*")#[0:100]

njet = infile["Events"]["nJet"].array()#[0:100]

In [ ]:
min_njets = 5
max_njets = 7

mask_njets = (njet>=min_njets) & (njet<=max_njets)

jets = ak.zip({
    "pt": jet_branch_arrays[mask_njets]["Jet_pt"],
    "phi": jet_branch_arrays[mask_njets]["Jet_phi"],
    "eta": jet_branch_arrays[mask_njets]["Jet_eta"],
    "mass": jet_branch_arrays[mask_njets]["Jet_mass"],
    "btag": jet_branch_arrays[mask_njets]["Jet_btagDeepB"],
}, with_name="Momentum4D")

In [ ]:
had_combos = ak.combinations(jets, 5)
#quad_charge = quads["0"].charge + quads["1"].charge + quads["2"].charge + quads["3"].charge

pt_cut = 25

mask_pt = (had_combos["0"].pt>pt_cut) & (had_combos["1"].pt>pt_cut) & (had_combos["2"].pt>pt_cut) \
          & (had_combos["3"].pt>pt_cut) & (had_combos["4"].pt>pt_cut)


jet1, jet2, jet3, jet4, jet5 = ak.unzip(had_combos[mask_pt])

p4_had = jet1 + jet2 + jet3
p4_bnv = jet4 + jet5

#mask_pt = (jet1.pt>25) & (jet2.pt>25) & (jet3.pt>25) & (jet4.pt>25) & (jet5.pt>25)

mask_btag = (jet1.btag>0.5) | (jet2.btag>0.5) & (jet3.btag>0.5)



In [ ]:
plt.figure()
plt.hist(ak.flatten(jet1.pt),bins=25,range=(0,100));
plt.hist(ak.flatten(jet2.pt),bins=25,range=(0,100));
plt.hist(ak.flatten(jet3.pt),bins=25,range=(0,100));



In [ ]:
x = (jet1 + jet2 + jet3)#[mask_pt & mask_btag]

print(len(ak.flatten(x.mass)))

plt.hist(ak.flatten(x.mass), bins=100, range=(0, 600));

In [ ]:
njet

In [ ]:
for m in x.mass:
    print(m)

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.hist(ak.flatten((jet1 + jet2).mass), bins=100, range=(0, 600));

plt.subplot(1,3,2)
plt.hist(ak.flatten((jet1 + jet3).mass), bins=100, range=(0, 600));

plt.subplot(1,3,3)
plt.hist(ak.flatten((jet2 + jet3).mass), bins=100, range=(0, 600));